In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!pip install datasets
!pip install -q tf-models-official==2.3.0
#gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12"
#tf.io.gfile.listdir(gs_folder_bert)


     |████████████████████████████████| 186 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 243 kB 6.5 MB/s eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.55.1
    Uninstalling tqdm-4.55.1:
      Successfully uninstalled tqdm-4.55.1


In [37]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import albert,bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/albert/checkpoints/albert_v2_base"
tf.io.gfile.listdir(gs_folder_bert)


['30k-clean.model',
 '30k-clean.vocab',
 'albert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index']

In [52]:
# Loading the mrpc dataset
original = load_dataset('glue', 'mrpc')
#original, info = tfds.load('glue/mrpc', with_info=True,batch_size=-1)

In [53]:
# Spliting the datasets into test train and validation
#original = dataset['train'].train_test_split(test_size=0.1, seed=1)
train = pd.DataFrame(original['train'])  
val = pd.DataFrame(original['validation'])
test = pd.DataFrame(original['test']) 
print(train.shape)
print(val.shape)
print(test.shape)

(3668, 4)
(408, 4)
(1725, 4)


In [54]:
#Setup Tokeniser
bert_model="albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(bert_model)
print(len(tokenizer.vocab))


30000


In [56]:
# Encode sentences
def encode_sentence(s, tokenizer):
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(glue_dict, tokenizer):
    num_examples = len(glue_dict["sentence1"])
    sentence1 = tf.ragged.constant([encode_sentence(s, tokenizer) for s in np.array(glue_dict["sentence1"])])
    sentence2 = tf.ragged.constant([encode_sentence(s, tokenizer) for s in np.array(glue_dict["sentence2"])])

    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=-1).to_tensor()

    inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

    return inputs

mrpc_train = bert_encode(train, tokenizer)
mrpc_train_labels = train['label']

mrpc_validation = bert_encode(val, tokenizer)
mrpc_validation_labels = val['label']

mrpc_test = bert_encode(test, tokenizer)
mrpc_test_labels  = test['label']


 

In [85]:
# Model Configuration
import json

albert_config_file = os.path.join(gs_folder_bert, "albert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(albert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

config_dict

{'attention_probs_dropout_prob': 0,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0,
 'embedding_size': 128,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'num_hidden_groups': 1,
 'net_structure_type': 0,
 'gap_size': 0,
 'num_memory_blocks': 0,
 'inner_group_num': 1,
 'down_scale_factor': 1,
 'type_vocab_size': 2,
 'vocab_size': 30000}

In [86]:
# Clasifier and encoder
bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=2)

In [87]:
# Set up epochs and steps
epochs = 10
batch_size = 16
eval_batch_size = 16

train_data_size = len(mrpc_train_labels)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(optimizer_type='adamw',
   init_lr =  2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

In [88]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bert_classifier.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

bert_classifier.fit(
      mrpc_train, mrpc_train_labels,
      validation_data=(mrpc_validation, mrpc_validation_labels),
      batch_size=16,
      epochs=epochs)

Epoch 1/10
230/230 [==============================] - 89s 326ms/step - loss: 0.6438 - accuracy: 0.6706 - val_loss: 0.6344 - val_accuracy: 0.6838
Epoch 2/10
230/230 [==============================] - 74s 321ms/step - loss: 0.6264 - accuracy: 0.6644 - val_loss: 0.6198 - val_accuracy: 0.6912
Epoch 3/10
230/230 [==============================] - 74s 321ms/step - loss: 0.5784 - accuracy: 0.7199 - val_loss: 0.6493 - val_accuracy: 0.6667
Epoch 4/10
230/230 [==============================] - 74s 320ms/step - loss: 0.5289 - accuracy: 0.7339 - val_loss: 0.6686 - val_accuracy: 0.6176
Epoch 5/10
230/230 [==============================] - 74s 320ms/step - loss: 0.4869 - accuracy: 0.7745 - val_loss: 0.7131 - val_accuracy: 0.6250
Epoch 6/10
230/230 [==============================] - 74s 321ms/step - loss: 0.4097 - accuracy: 0.8199 - val_loss: 0.7269 - val_accuracy: 0.6667
Epoch 7/10
230/230 [==============================] - 74s 321ms/step - loss: 0.3734 - accuracy: 0.8369 - val_loss: 0.8012 - val_ac

In [89]:
export_dir='./SavedModel.pb'
tf.saved_model.save(bert_classifier, export_dir=export_dir)

# Data is stored as .pb file by default


In [ ]:
## Taking user input and storing the tokenised tensors in a textfile to load in Java later
#sent1 = input()
#sent2 = input()

sent1  = "They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added ."
sent2 = "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale ."

my_examples = bert_encode(
    glue_dict = {
        'sentence1':[str(sent1)],
        'sentence2':[str(sent2)]
    },
    tokenizer=tokenizer)



input_word_ids = my_examples['input_word_ids'][0].numpy().astype(int)
input_mask = my_examples['input_mask'][0].numpy().astype(int)
input_type_ids  = my_examples['input_type_ids'][0].numpy().astype(int)
np.savetxt('user_input.txt', (input_word_ids.astype(int),input_mask.astype(int),input_type_ids.astype(int)),fmt='%d') 

# the "user_input.txt" file is later stored in java folder

In [ ]:
!saved_model_cli show --dir './SavedModel/' --all

In [ ]:
'''
MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_mask'] tensor_info:
        dtype: DT_INT32
        shape: (-1, -1)
        name: serving_default_input_mask:0
    inputs['input_type_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, -1)
        name: serving_default_input_type_ids:0
    inputs['input_word_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, -1)
        name: serving_default_input_word_ids:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classification'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')]
        Argument #2
          DType: bool
          Value: False
        Argument #3
          DType: NoneType
          Value: None
    Option #2
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/2')]
        Argument #2
          DType: bool
          Value: False
        Argument #3
          DType: NoneType
          Value: None
    Option #3
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/2')]
        Argument #2
          DType: bool
          Value: True
        Argument #3
          DType: NoneType
          Value: None
    Option #4
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')]
        Argument #2
          DType: bool
          Value: True
        Argument #3
          DType: NoneType
          Value: None

  Function Name: '_default_save_signature'
    Option #1
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')]

  Function Name: 'call_and_return_all_conditional_losses'
    Option #1
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')]
        Argument #2
          DType: bool
          Value: False
        Argument #3
          DType: NoneType
          Value: None
    Option #2
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')]
        Argument #2
          DType: bool
          Value: True
        Argument #3
          DType: NoneType
          Value: None
    Option #3
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/2')]
        Argument #2
          DType: bool
          Value: False
        Argument #3
          DType: NoneType
          Value: None
    Option #4
      Callable with:
        Argument #1
          DType: list
          Value: [TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/2')]
        Argument #2
          DType: bool
          Value: True
        Argument #3
          DType: NoneType
          Value: None
'''